In [1]:
import sys
import os

# Add the path to the aima-python repo to sys.path
sys.path.append(os.path.abspath('/Users/lukegosnell/Downloads/CUYear5/AI/FinalProject/aima_python'))

from aima_python.games4e import Game
from aima_python.utils4e import vector_add, MCT_Node, ucb
from collections import defaultdict

import random
random.seed(109)

In [16]:
class Mancala(Game):
    """Mancala game implementation."""

    def __init__(self):
        self.squares = {i for i in range(14) if i != 6 and i != 13}  # Indices of pits (excluding mancalas)
        self.initial = Board(pits=[4] * 6 + [0] + [4] * 6 + [0], to_move='P1')

    def actions(self, board):
        """Return a list of legal moves (non-empty pits on the current player's side)."""
        start, end = (0, 6) if board.to_move == 'P1' else (7, 13)
        return [i for i in range(start, end) if board.pits[i] > 0]

    def result(self, board, action):
        """Apply the move (distributing stones) and return the new board state."""
        pits = board.pits[:]
        player = board.to_move
        stones = pits[action]
        pits[action] = 0

        idx = action
        while stones > 0:
            idx = (idx + 1) % 14
            if (player == 'P1' and idx == 13) or (player == 'P2' and idx == 6):
                continue  # Skip opponent's mancala
            pits[idx] += 1
            stones -= 1

        # Capture condition
        if (player == 'P1' and 0 <= idx < 6 or player == 'P2' and 7 <= idx < 13) \
                and pits[idx] == 1 and pits[12 - idx] > 0:
            pits[6 if player == 'P1' else 13] += pits[idx] + pits[12 - idx]
            pits[idx] = pits[12 - idx] = 0

        # Determine next player
        next_to_move = player if (player == 'P1' and idx == 6 or player == 'P2' and idx == 13) else ('P2' if player == 'P1' else 'P1')

        return board.new({'pits': pits, 'to_move': next_to_move})

    def utility(self, board, player):
        """Return the game utility for the given player."""
        if self.is_terminal(board):
            score_p1 = sum(board.pits[:7])  # P1's Mancala score
            score_p2 = sum(board.pits[7:])  # P2's Mancala score
            if player == 'P1':
                return score_p1 - score_p2  # Max - Min for P1
            else:
                return score_p2 - score_p1  # Max - Min for P2
            return 0

    def is_terminal(self, board):
        """Check if the game has ended."""
        return all(p == 0 for p in board.pits[:6]) or all(p == 0 for p in board.pits[7:13])

    def display(self, board, current_player=None, move_from=None):
        """Display the board state."""
        
        print("  " + " ".join(map(str, board.pits[12:6:-1])))
        print(f"{board.pits[13]}                  {board.pits[6]}")
        print("  " + " ".join(map(str, board.pits[:6])))
        print("\n")

        if current_player is not None and move_from is not None:
            print(f"Player {current_player} moved from pit {move_from}")

In [17]:
class Board(defaultdict):
    """A Mancala board with pits and a player to move."""

    def __init__(self, pits=None, to_move=None, **kwds):
        super().__init__(int)
        self.pits = pits or [0] * 14
        self.to_move = to_move

    def new(self, changes: dict, **kwds) -> 'Board':
        """Create a new board state with the specified changes."""
        board = Board(pits=self.pits[:], to_move=self.to_move, **kwds)
        board.__dict__.update(changes)
        return board

    def __hash__(self):
        return hash(tuple(self.pits)) + hash(self.to_move)

    def __repr__(self):
        return f"Mancala({self.pits}, {self.to_move})"

In [18]:
# Example game simulation
from random import choice

def random_player(game, state):
    return choice(game.actions(state))

def play_game(game, players, verbose=False):
    """Simulate a game between two players"""
    state = game.initial
    while not game.is_terminal(state):
        current_player = players[state.to_move]  # Get the current player
        action = current_player(game, state)  # Get the player's move
        state = game.result(state, action)  # Apply the move and get the new state
        
        if verbose:
            game.display(state)  # Display the board state if verbose is True
    
    return game.utility(state, 'P1')  # Return the utility of the game for player 'P1'

In [19]:
def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

In [20]:
def alphabeta(game, state, alpha=-float('inf'), beta=float('inf')):
    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, 'P1'), None
        best_action = None
        v = -float('inf')
        for action in game.actions(state):
            v2, _ = min_value(game.result(state, action), alpha, beta)
            if v2 > v:
                v, best_action = v2, action
            if v >= beta:
                return v, best_action
            alpha = max(alpha, v)
        return v, best_action

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, 'P2'), None
        best_action = None
        v = float('inf')
        for action in game.actions(state):
            v2, _ = max_value(game.result(state, action), alpha, beta)
            if v2 < v:
                v, best_action = v2, action
            if v <= alpha:
                return v, best_action
            beta = min(beta, v)
        return v, best_action

    return max_value(state, alpha, beta)[1]


In [21]:
def minimax(game, state):
    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, 'P1')
        v = -float('inf')
        for action in game.actions(state):
            v = max(v, min_value(game.result(state, action)))
        return v

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, 'P2')
        v = float('inf')
        for action in game.actions(state):
            v = min(v, max_value(game.result(state, action)))
        return v

    # The minimax search returns the best action for the player whose turn it is.
    best_action = None
    best_value = -float('inf') if state.to_move == 'P1' else float('inf')
    
    for action in game.actions(state):
        if state.to_move == 'P1':
            value = min_value(game.result(state, action))
            if value > best_value:
                best_value, best_action = value, action
        else:
            value = max_value(game.result(state, action))
            if value < best_value:
                best_value, best_action = value, action

    return best_action


In [ ]:
# Create a Mancala game and players
players = {
    'P1': random_player,  # Random player for 'P1'
    'P2': minimax  # Player using alphabeta_search for 'P2'
}

# Play the game and print the result for 'P1'
result = play_game(Mancala(), players, verbose=True)

print(f"Game over! Result for 'P1': {result}")


  4 4 4 4 4 4
0                  0
  4 0 5 5 5 5


